This notebook performs the preprocessing of the raw data.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

import pandas as pd
import numpy as np

from src.preprocess import parse_all_data, clean_raw_data, computeTimeDeltaTraces

import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "none"

import src.colors as colors
from src.helpers import Modes, Parameters

In [ ]:
# Clean up and bug correction of the data collected by Anna
# clean_raw_data()

In [4]:
# Create and save the median values for faster plotting of the different settings
computeTimeDeltaTraces()

Processed data retrieved.
Done with TimeDeltaTraces_A_0_(nan).csv
Done with TimeDeltaTraces_A_0_(-35.0).csv
Done with TimeDeltaTraces_A_0_(-34.0).csv
Done with TimeDeltaTraces_A_0_(-33.0).csv
Done with TimeDeltaTraces_A_0_(-32.0).csv
Done with TimeDeltaTraces_A_0_(-16.0).csv
Done with TimeDeltaTraces_A_0_(-19.0).csv
Done with TimeDeltaTraces_A_0_(-23.0).csv
Done with TimeDeltaTraces_A_0_(-29.0).csv
Done with TimeDeltaTraces_A_0_(-30.0).csv
Done with TimeDeltaTraces_A_0_(-31.0).csv
Done with TimeDeltaTraces_A_0_(-14.0).csv
Done with TimeDeltaTraces_A_0_(-11.0).csv
Done with TimeDeltaTraces_A_0_(-7.0).csv
Done with TimeDeltaTraces_A_0_(-4.0).csv
Done with TimeDeltaTraces_A_0_(-3.0).csv
Done with TimeDeltaTraces_A_0_(-2.0).csv
Done with TimeDeltaTraces_A_0_(-1.0).csv
Done with TimeDeltaTraces_A_0_(0.0).csv
Done with TimeDeltaTraces_A_0_(1.0).csv
Done with TimeDeltaTraces_A_0_(16.0).csv
Done with TimeDeltaTraces_A_0_(13.0).csv


/home/user/miniconda3/envs/DashTuto/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3334: RuntimeWarning:

Mean of empty slice.

/home/user/miniconda3/envs/DashTuto/lib/python3.8/site-packages/numpy/core/_methods.py:161: RuntimeWarning:

invalid value encountered in double_scalars



Done with TimeDeltaTraces_A_0_(9.0).csv
Done with TimeDeltaTraces_A_0_(5.0).csv
Done with TimeDeltaTraces_A_0_(4.0).csv
Done with TimeDeltaTraces_A_0_(3.0).csv
Done with TimeDeltaTraces_A_0_(2.0).csv
Done with TimeDeltaTraces_A_0_(-10.0).csv
Done with TimeDeltaTraces_A_0_(14.0).csv
Done with TimeDeltaTraces_A_0_(11.0).csv
Done with TimeDeltaTraces_A_0_(8.0).csv
Done with TimeDeltaTraces_A_0_(-8.0).csv
Done with TimeDeltaTraces_A_0_(15.0).csv
Done with TimeDeltaTraces_A_0_(12.0).csv
Done with TimeDeltaTraces_A_0_(19.0).csv
Done with TimeDeltaTraces_A_0_(6.0).csv
Done with TimeDeltaTraces_A_0_(-13.0).csv
Done with TimeDeltaTraces_A_0_(-9.0).csv
Done with TimeDeltaTraces_A_0_(-5.0).csv
Done with TimeDeltaTraces_A_0_(7.0).csv
Done with TimeDeltaTraces_A_0_(-12.0).csv
Done with TimeDeltaTraces_A_0_(-15.0).csv
Done with TimeDeltaTraces_A_0_(-6.0).csv
Done with TimeDeltaTraces_A_0_(20.0).csv
Done with TimeDeltaTraces_A_0_(17.0).csv
Done with TimeDeltaTraces_A_0_(10.0).csv
Done with TimeDeltaT

Done with TimeDeltaTraces_B_1_(11.0).csv
Done with TimeDeltaTraces_B_1_(8.0).csv
Done with TimeDeltaTraces_B_1_(-8.0).csv
Done with TimeDeltaTraces_B_1_(15.0).csv
Done with TimeDeltaTraces_B_1_(12.0).csv
Done with TimeDeltaTraces_B_1_(19.0).csv
Done with TimeDeltaTraces_B_1_(6.0).csv
Done with TimeDeltaTraces_B_1_(-13.0).csv
Done with TimeDeltaTraces_B_1_(-9.0).csv
Done with TimeDeltaTraces_B_1_(-5.0).csv
Done with TimeDeltaTraces_B_1_(7.0).csv
Done with TimeDeltaTraces_B_1_(-12.0).csv
Done with TimeDeltaTraces_B_1_(-15.0).csv
Done with TimeDeltaTraces_B_1_(-6.0).csv
Done with TimeDeltaTraces_B_1_(20.0).csv
Done with TimeDeltaTraces_B_1_(17.0).csv
Done with TimeDeltaTraces_B_1_(10.0).csv
Done with TimeDeltaTraces_B_1_(22.0).csv
Done with TimeDeltaTraces_B_1_(-20.0).csv
Done with TimeDeltaTraces_B_1_(-17.0).csv
Done with TimeDeltaTraces_B_1_(-25.0).csv
Done with TimeDeltaTraces_B_1_(-22.0).csv
Done with TimeDeltaTraces_B_1_(-18.0).csv
Done with TimeDeltaTraces_B_1_(-26.0).csv
Done with 

In [10]:
df = parse_all_data()
# TransPair = 'A'

# for SamePayload in df['SamePayload'].unique():
#     for PowerDelta in df['PowerDelta'].dropna().unique():
#         file_name = 'TimeDeltaTraces_%s_%i_(%i).csv' % (TransPair,SamePayload,PowerDelta)
#         print(file_name)
                

Processed data retrieved.
TimeDeltaTraces_A_0_(-35).csv
TimeDeltaTraces_A_0_(-34).csv
TimeDeltaTraces_A_0_(-33).csv
TimeDeltaTraces_A_0_(-32).csv
TimeDeltaTraces_A_0_(-16).csv
TimeDeltaTraces_A_0_(-19).csv
TimeDeltaTraces_A_0_(-23).csv
TimeDeltaTraces_A_0_(-29).csv
TimeDeltaTraces_A_0_(-30).csv
TimeDeltaTraces_A_0_(-31).csv
TimeDeltaTraces_A_0_(-14).csv
TimeDeltaTraces_A_0_(-11).csv
TimeDeltaTraces_A_0_(-7).csv
TimeDeltaTraces_A_0_(-4).csv
TimeDeltaTraces_A_0_(-3).csv
TimeDeltaTraces_A_0_(-2).csv
TimeDeltaTraces_A_0_(-1).csv
TimeDeltaTraces_A_0_(0).csv
TimeDeltaTraces_A_0_(1).csv
TimeDeltaTraces_A_0_(16).csv
TimeDeltaTraces_A_0_(13).csv
TimeDeltaTraces_A_0_(9).csv
TimeDeltaTraces_A_0_(5).csv
TimeDeltaTraces_A_0_(4).csv
TimeDeltaTraces_A_0_(3).csv
TimeDeltaTraces_A_0_(2).csv
TimeDeltaTraces_A_0_(-10).csv
TimeDeltaTraces_A_0_(14).csv
TimeDeltaTraces_A_0_(11).csv
TimeDeltaTraces_A_0_(8).csv
TimeDeltaTraces_A_0_(-8).csv
TimeDeltaTraces_A_0_(15).csv
TimeDeltaTraces_A_0_(12).csv
TimeDeltaTra

In [ ]:
# Compute (or retrieve) all preprocessed data
df = parse_all_data(force_computation='prr')

In [ ]:
a = df.PowerDelta.dropna().unique().tolist()
# print(sorted(a))

b = range(-16,17)
print([x for x in b])

In [4]:
# Compute (or retrieve) all preprocessed data
df = parse_all_data()

Processed data retrieved.


In [15]:
# Select the data to plot 
# (for now we make it simple...)

filter = (
#     (df['DateTime'] == "2019-10-04 13:55:00") & 
    (df['TransPair'] == "B") & 
    (df['SamePayload'] == 1) & 
    (df["PowerDelta"] == 9)  
#     (8.5 <= df["PowerDelta"])
)
filtered_df = df.where(filter).dropna()
# display(filtered_df)

In [ ]:
# 

In [61]:
figure = go.Figure()

# print(Modes)

for mode in Modes:
    
    mode_filter = (filtered_df["Mode"] == Modes[mode]['id'])
    mode_df = filtered_df.where(mode_filter).dropna()
    
    # Force displaying the trace, even if empty
    if len(mode_df) == 0:
        x_data = [np.nan]
        y_data = [np.nan]
        x_median = [np.nan]
        y_median = [np.nan]
    else: 
        x_data = mode_df["TimeDelta"]
        y_data = mode_df["PRR"]
        
        # Compute the median line
        x_median = sorted(set(x_data))
        y_median = []
        for x in x_median:
            median_filter = (mode_df["TimeDelta"] == x)
            median_data = mode_df.where(median_filter).dropna().PRR
            y_median.append(np.median(median_data))
    
    scatter = go.Scatter(
        x=x_data,
        y=y_data,
        mode='markers',
        marker={'color':Modes[mode]['color']},
        showlegend=True,
        legendgroup=Modes[mode]['id'],
        name=Modes[mode]['label'],
        opacity=0.7,
#         fill='toself',
        hoverinfo='skip',
#         line=dict(color=colors.light_orange, width=0),
    )
    figure.add_trace(scatter)
    
    
    median_line = go.Scatter(
        x=x_median,
        y=y_median,
        mode='lines',
        line={'color':Modes[mode]['color']},
        showlegend=False,
        legendgroup=Modes[mode]['id'],
        name=Modes[mode]['label']+' median',
#         fill='toself',
#         hoverinfo='skip',
#         line=dict(color=colors.light_orange, width=0),
    )
    figure.add_trace(median_line)
    
    
    
custom_layout = {
    "title":{
        'text':'PRR = f(Time Delta) <br> with Power Delta = %d dB' % 0,
    },
#     "width":linewidth_px,
#     "height":220,
#     "font":{"size":font_size},
    "margin":dict(t=150),
    "xaxis":{
        'title':{'text':'Transmitters Time Delta [ticks]'},
            },
    "yaxis":{
        'range':[0,103]
    },
    "legend":{
#         'font':{'size':font_size},
        'orientation':"h",
        'x': 0.5,
        'y':1.15,
        'xanchor':"center",
        'yanchor':"top",
    }
}
figure.update_layout(custom_layout)
figure.show()